# AIML Online Capstone - AUTOMATIC TICKET ASSIGNMENT

Pre-Processing, Data Visualization and EDA
● Exploring the given Data files
● Understanding the structure of data
● Missing points in data
● Finding inconsistencies in the data
● Visualizing different patterns
● Visualizing different text features
● Dealing with missing values
● Text preprocessing
● Creating word vocabulary from the corpus of report text data
● Creating tokens as required

In [0]:
# Initialize the random number generator
import random
random.seed(0)

# Ignore the warnings
import warnings
warnings.filterwarnings("ignore")

In [0]:
from google.colab import drive
drive.mount('/content/drive/')

Drive already mounted at /content/drive/; to attempt to forcibly remount, call drive.mount("/content/drive/", force_remount=True).


In [0]:
#Set your project path 
import os
project_path =os.chdir('/content/drive/My Drive/Colab Notebooks/Capstone') 

In [0]:

import numpy as np
import pandas as pd
import tensorflow as tf
#from tensorflow.contrib.rnn import BasicLSTMCell
from tensorflow.python.ops.rnn import bidirectional_dynamic_rnn as bi_rnn
import time
import os
from sklearn.model_selection import train_test_split

In [0]:
df = pd.read_excel (r'input_data.xlsx')

In [0]:
df.head()

,Short description,Description,Caller,Assignment group
0,login issue,-verified user details.(employee# & manager na...,spxjnwir pjlcoqds,GRP_0
1,outlook,\r\n\r\nreceived from: hmjdrvpb.komuaywn@gmail...,hmjdrvpb komuaywn,GRP_0
2,cant log in to vpn,\r\n\r\nreceived from: eylqgodm.ybqkwiam@gmail...,eylqgodm ybqkwiam,GRP_0
3,unable to access hr_tool page,unable to access hr_tool page,xbkucsvz gcpydteq,GRP_0
4,skype error,skype error,owlgqjme qhcozdfx,GRP_0


In [0]:
df.describe()

,Short description,Description,Caller,Assignment group
count,8492,8499,8500,8500
unique,7481,7817,2950,74
top,password reset,the,bpctwhsn kzqsbmtp,GRP_0
freq,38,56,810,3976


In [0]:
df.isnull().sum()

Short description    8
Description          1
Caller               0
Assignment group     0
dtype: int64

In [0]:
nan_rows1 = df[df['Short description'].isnull()]
nan_rows2 = df[df['Description'].isnull()]

In [0]:
nan_rows1

,Short description,Description,Caller,Assignment group
2604,NaN,\r\n\r\nreceived from: ohdrnswl.rezuibdt@gmail...,ohdrnswl rezuibdt,GRP_34
3383,NaN,\r\n-connected to the user system using teamvi...,qftpazns fxpnytmk,GRP_0
3906,NaN,-user unable tologin to vpn.\r\n-connected to...,awpcmsey ctdiuqwe,GRP_0
3910,NaN,-user unable tologin to vpn.\r\n-connected to...,rhwsmefo tvphyura,GRP_0
3915,NaN,-user unable tologin to vpn.\r\n-connected to...,hxripljo efzounig,GRP_0
3921,NaN,-user unable tologin to vpn.\r\n-connected to...,cziadygo veiosxby,GRP_0
3924,NaN,name:wvqgbdhm fwchqjor\nlanguage:\nbrowser:mic...,wvqgbdhm fwchqjor,GRP_0
4341,NaN,\r\n\r\nreceived from: eqmuniov.ehxkcbgj@gmail...,eqmuniov ehxkcbgj,GRP_0


In [0]:
nan_rows2

,Short description,Description,Caller,Assignment group
4395,i am locked out of skype,NaN,viyglzfo ajtfzpkb,GRP_0


In [0]:
df.loc[df['Description'].isnull(),'Description'] = df['Short description']

In [0]:
df.loc[[4395]]

,Short description,Description,Caller,Assignment group
4395,i am locked out of skype,i am locked out of skype,viyglzfo ajtfzpkb,GRP_0


In [0]:
df.loc[df['Short description'].isnull(),'Short description'] = df['Description']

In [0]:
df.loc[[3924]]

,Short description,Description,Caller,Assignment group
3924,name:wvqgbdhm fwchqjor\nlanguage:\nbrowser:mic...,name:wvqgbdhm fwchqjor\nlanguage:\nbrowser:mic...,wvqgbdhm fwchqjor,GRP_0


In [0]:
df.isnull().sum()

Short description    0
Description          0
Caller               0
Assignment group     0
dtype: int64

Preprocess text

In [0]:
# Merge 'Short description' and 'Description' columns into single column 'description'
df['description'] = (df['Short description']+df['Description']).map(str)

In [0]:
df_Tickets = df.drop(['Description'], axis=1)

In [0]:
df_Tickets.head()

,Short description,Caller,Assignment group,description
0,login issue,spxjnwir pjlcoqds,GRP_0,login issue-verified user details.(employee# &...
1,outlook,hmjdrvpb komuaywn,GRP_0,outlook\r\n\r\nreceived from: hmjdrvpb.komuayw...
2,cant log in to vpn,eylqgodm ybqkwiam,GRP_0,cant log in to vpn\r\n\r\nreceived from: eylqg...
3,unable to access hr_tool page,xbkucsvz gcpydteq,GRP_0,unable to access hr_tool pageunable to access ...
4,skype error,owlgqjme qhcozdfx,GRP_0,skype error skype error


In [0]:
df_Tickets = df_Tickets.drop(['Short description'], axis=1)

In [0]:
# Solution

import re
temp =[]
snow = nltk.stem.SnowballStemmer('english')
for sentence in df_Tickets['description']:
    sentence = sentence.lower()                 # Converting to lowercase
    cleanr = re.compile('<.*?>')
    sentence = re.sub(cleanr, ' ', sentence)        #Removing HTML tags
    sentence = re.sub(r'[?|!|\'|"|#|@|&|$]',r'',sentence)
    sentence = re.sub(r'[.|,|)|(|\|/]',r' ',sentence)        #Removing Punctuations
    
    words = [snow.stem(word) for word in sentence.split() if word not in stopwords.words('english')]   # Stemming and removing stopwords
    temp.append(words)
    
df_Tickets['Descrip'] = temp    

In [0]:
df_Tickets = df_Tickets.drop(['description'], axis=1)
df_Tickets.head()

,Caller,Assignment group,Descrip
0,spxjnwir pjlcoqds,GRP_0,"[login, issue-verifi, user, detail, employe, m..."
1,hmjdrvpb komuaywn,GRP_0,"[outlook, receiv, from:, hmjdrvpb, komuaywngma..."
2,eylqgodm ybqkwiam,GRP_0,"[cant, log, vpn, receiv, from:, eylqgodm, ybqk..."
3,xbkucsvz gcpydteq,GRP_0,"[unabl, access, hr_tool, pageun, access, hr_to..."
4,owlgqjme qhcozdfx,GRP_0,"[skype, error, skype, error]"


In [0]:
df_Tickets['Caller'].nunique()

2950

In [0]:
df_Tickets.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8500 entries, 0 to 8499
Data columns (total 3 columns):
 #   Column            Non-Null Count  Dtype 
---  ------            --------------  ----- 
 0   Caller            8500 non-null   object
 1   Assignment group  8500 non-null   object
 2   Descrip           8500 non-null   object
dtypes: object(3)
memory usage: 199.3+ KB


In [0]:
df_Tickets['Assignment group'].nunique()

74